In [7]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc

import numpy as np
import pandas as pd
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
from fracridge import FracRidgeRegressorCV
from sklearn.model_selection import KFold

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from research.data.natural_scenes import (
    NaturalScenesDataset,
    StimulusDataset,
    KeyDataset
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
dataset_path = Path('D:\\Datasets\\NSD\\')
nsd = NaturalScenesDataset(dataset_path)

derivatives_path = dataset_path / 'derivatives'
betas_path = dataset_path / 'nsddata_betas' / 'ppdata'
ppdata_path = dataset_path / 'nsddata' / 'ppdata'

In [9]:
# Load image ids of the shared 1000 images across all participants
shared_1000_path = dataset_path / 'nsddata' / 'stimuli' / 'nsd' / 'shared1000.tsv'
shared_1000 = pd.read_csv(shared_1000_path, sep='\t', header=None)
shared_1000 = set(shared_1000[0])

In [12]:
# Define a train-test-validation split
split_name = 'split-01'
N_test = 1000
N_validation = 1000

seed = 0

for subject_name, subject_data in nsd.subjects.items():
    responses = subject_data['responses']
    
    image_ids = responses['73KID'].to_numpy()
    unique_image_ids, unique_counts = np.unique(image_ids, return_counts=True)
    three_repetition_ids = unique_image_ids[unique_counts == 3]
    subject_data['three_repetition_ids'] = set(three_repetition_ids)
    print(f'{subject_name} {image_ids.shape=}, {len(three_repetition_ids)=}')
    
shared_1000_three_repetitions = set.intersection(
    shared_1000,
    *[subject_data['three_repetition_ids']
    for subject_data in nsd.subjects.values()]
)
print(f'{len(shared_1000_three_repetitions)=}')
N_non_shared = N_test - len(shared_1000_three_repetitions)

print(asdf)

for subject_name, subject_data in subjects.items():
    three_repetition_ids = subject_data['three_repetition_ids']
    non_shared_three_repetition_ids = list(three_repetition_ids - shared_1000_three_repetitions)
    random.Random(seed).shuffle(non_shared_three_repetition_ids)
    
    test_image_ids = list(shared_1000_three_repetitions) + non_shared_three_repetition_ids[:N_non_shared]
    validation_image_ids = non_shared_three_repetition_ids[N_non_shared:(N_non_shared + N_validation)]
    subject_data['test_image_ids'] = np.array(test_image_ids)
    subject_data['validation_image_ids'] = np.array(test_image_ids)
    
    test_image_ids = set(test_image_ids)
    validation_image_ids = set(validation_image_ids)
    image_ids = subject_data['responses']['73KID'].to_numpy()
    subject_data['test_response_ids'] = np.argwhere([image_id in test_image_ids for image_id in image_ids])[:, 0]
    subject_data['validation_response_ids'] = np.argwhere([image_id in validation_image_ids for image_id in image_ids])[:, 0]
    
with h5py.File(derivatives_path / 'data_splits' / f'{split_name}.hdf5', 'w') as f:
    for subject_name, subject_data in subjects.items():
        subject = f.require_group(subject_name)
        
        three_repetition_ids = subject_data['three_repetition_ids']
        non_shared_three_repetition_ids = list(three_repetition_ids - shared_1000_three_repetitions)
        random.Random(seed).shuffle(non_shared_three_repetition_ids)

        test_image_ids = list(shared_1000_three_repetitions) + non_shared_three_repetition_ids[:N_non_shared]
        validation_image_ids = non_shared_three_repetition_ids[N_non_shared:(N_non_shared + N_validation)]
        subject['test_image_ids'] = np.array(test_image_ids)
        subject['validation_image_ids'] = np.array(test_image_ids)

        test_image_ids = set(test_image_ids)
        validation_image_ids = set(validation_image_ids)
        image_ids = subject_data['responses']['73KID'].to_numpy()
        subject['test_response_mask'] = np.array([image_id in test_image_ids for image_id in image_ids], dtype=bool)
        subject['validation_response_mask'] = np.array([image_id in validation_image_ids for image_id in image_ids], dtype=bool)


subj01 image_ids.shape=(27750,), len(three_repetition_ids)=8420
subj02 image_ids.shape=(27750,), len(three_repetition_ids)=8420
subj03 image_ids.shape=(21750,), len(three_repetition_ids)=5081
subj04 image_ids.shape=(20250,), len(three_repetition_ids)=4424
subj05 image_ids.shape=(27750,), len(three_repetition_ids)=8420
subj06 image_ids.shape=(21750,), len(three_repetition_ids)=5081
subj07 image_ids.shape=(27750,), len(three_repetition_ids)=8420
subj08 image_ids.shape=(20250,), len(three_repetition_ids)=4424
len(shared_1000_three_repetitions)=413


NameError: name 'asdf' is not defined